In [1]:
from datetime import datetime, timedelta

import ccxt
import numpy as np
import pandas as pd

import plotly.graph_objects as go

ex = ccxt.binance()

In [2]:
symbols = [line.strip() for line in open("symbols.txt", "r").readlines()]

In [3]:
def get_ohlcv(symbol, days_count, timeframe):
    from_ts = int((datetime.today() - timedelta(days=days_count)).timestamp() * 1000)

    ts = []
    time = []
    open = []
    high = []
    low = []
    close = []
    volume = []

    mid = []
    open_close_dif = []
    low_high_dif = []
    open_close_dif_abs = []
    low_high_dif_abs = []

    ohlcvs = ex.fetch_ohlcv(symbol, timeframe, since=from_ts)

    for ohlcv in ohlcvs:
        t, o, h, l, c, v = ohlcv
        ts.append(t)
        time.append(pd.to_datetime(t, unit='ms'))
        open.append(o)
        high.append(h)
        low.append(l)
        close.append(c)
        volume.append(v)

        mid.append((h+l) / 2)
        open_close_dif.append(o-c)
        low_high_dif.append(l-h)
        open_close_dif_abs.append(abs(o-c))
        low_high_dif_abs.append(abs(l-h))

    return pd.DataFrame({"ts": ts, "time": time, "open": open, "high": high, "low": low, "close": close, "volume": volume, "mid": mid, "open_close_dif": open_close_dif, "low_high_dif": low_high_dif, "open_close_dif_abs": open_close_dif_abs, "low_high_dif_abs": low_high_dif_abs})

In [4]:
def fill_ohcls(ohcls, log_price = True):
    diff = np.abs(np.diff(ohcls['mid']))

    mid_shifted = ohcls['mid'][1:].to_numpy()
    mid_without_last = ohcls['mid'][:-1].to_numpy()

    if log_price:
        div = mid_shifted / mid_without_last
        log_diff = np.log(div)
    else:
        div = (mid_shifted - mid_without_last) / mid_shifted
        log_diff = div

    ohcls.loc[1:, ['diff']] = diff
    ohcls.loc[1:, ['log_diff']] = np.abs(log_diff)

    return ohcls[1:]

In [14]:
def plot_volatility(ohcls, annualized=False):
    volatility = go.Scatter(x=ohcls['time'], y=ohcls["log_diff"], name = 'price_diff', mode='lines')

    time_period = ohcls.iloc[[0,-1]]['time']
    mean = ohcls['log_diff'].mean()

    y_sigma1 = annualize(ohcls) if annualized else ohcls['log_diff'].std()
    y_sigma2 = y_sigma1 * 2
    y_sigma3 = y_sigma1 * 3

    mean = go.Scatter(x=time_period, y = [mean, mean], name = 'mean', mode='lines', line={'width': 1})
    sigma1 = go.Scatter(x=time_period, y = [y_sigma1, y_sigma1], name = 'sigma1', mode='lines', line={'width': 1})
    sigma2 = go.Scatter(x=time_period, y = [y_sigma2, y_sigma2], name = 'sigma2', mode='lines', line={'width': 1})
    sigma3 = go.Scatter(x=time_period, y = [y_sigma3, y_sigma3], name = 'sigma3', mode='lines', line={'width': 1})

    fig = go.Figure()
    fig.add_trace(volatility)
    fig.add_trace(mean)
    fig.add_trace(sigma1)
    fig.add_trace(sigma2)
    fig.add_trace(sigma3)
    fig.show()

In [6]:
def annualize(ohcls, symbol = 'None'):
    vol = ohcls['log_diff'].std()

    timestamps = ohcls['ts'].to_numpy()
    ts_start, ts_end = timestamps[0], timestamps[-1]
    dt_ms = ts_end - ts_start
    year_ms = 365 * 24 * 60 * 60 * 1000

    n_periods = year_ms / dt_ms
    annualized = vol * np.sqrt(n_periods)

    # print(symbol, 'period:', int((dt_ms / 1000 / 60 / 60 / 24) * 100)/100, 'days')

    return annualized


In [7]:
def create_ohlcv_df(symbol, days_count, timeframe, log_price = True):
    """
    start volatility investigation
    :param str symbol: Unified CCXT market symbol
    :param int days_count: last days_count to fetch
    :param str timeframe: "5m","15m","30m","1h","2h","4h","6h","12h", or "1d"
    :param bool|None log_price: default True: use log(price_diff) or not
    """
    ohcls = get_ohlcv(symbol, days_count, timeframe)
    ohcls = fill_ohcls(ohcls, log_price)

    return ohcls

In [8]:
def start(symbols, days_count, timeframe):
    coins = []
    volatility = []

    for symbol in symbols:
        ohcls = create_ohlcv_df(symbol, days_count, timeframe)
        annualized_vol = annualize(ohcls, symbol)

        coins.append(symbol)
        volatility.append(annualized_vol)

    print(pd.DataFrame({"annual_vol": volatility}, index=coins))

In [13]:
def plot(symbol, days_count, timeframe, annualized = False):
    ohcls = create_ohlcv_df(symbol, days_count, timeframe)
    plot_volatility(ohcls, annualized)

In [10]:
start(symbols[:10], 5, '15m')

            annual_vol
FORTH/BUSD    0.021079
DOT/BUSD      0.011083
FIL/BUSD      0.014089
BTCST/BUSD    0.060941
SHIB/BUSD     0.013849
GMT/BUSD      0.013018
ICP/BUSD      0.012570
RUNE/BUSD     0.014900
APE/BUSD      0.016111
LINK/BUSD     0.014994


In [17]:
plot('SANTOS/BUSD', 5, '15m', annualized=False)